# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import pandas as pd
import gzip
from datetime import datetime, timedelta
from cloud_sql_connector import DBConfig, getconn
import pg8000
import sqlalchemy

In [ ]:
from dotenv import find_dotenv
from dotenv import load_dotenv
load_dotenv(find_dotenv())

### DB Connection 

In [ ]:
config = DBConfig(
    instance_name=os.getenv('LEAR_DB_CONNECTION_NAME'),
    database=os.getenv('DATABASE_NAME'),
    user=os.getenv('DATABASE_USER'),
    ip_type="public",
    schema="business"
    )

def get_conn():
    """Create a connection to Google Cloud SQL using the custom cloud-sql-connector."""
    return getconn(config)

print('LEAR_DB_CONNECTION_NAME:', os.getenv('LEAR_DB_CONNECTION_NAME'))
print('DATABASE_NAME:', os.getenv('DATABASE_NAME'))
print('DATABASE_USER:', os.getenv('DATABASE_USER'))
print('GOOGLE_APPLICATION_CREDENTIALS:', os.getenv('GOOGLE_APPLICATION_CREDENTIALS'))
print('DBConfig:', config)

engine = sqlalchemy.create_engine(
    "postgresql+pg8000://",
    creator=get_conn,
    )

print("Cloud SQL engine created successfully!")
print(f"Engine: {engine}")

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(sqlalchemy.text("SELECT 1 as test"))
        print("✅ Database connection test successful!")
        print(f"Test result: {result.fetchone()}")
except Exception as e:
    print(f"❌ Connection failed: {e}")



### Collect data

In [ ]:
icbc_query = """
select
 CASE WHEN state='ACTIVE' THEN '1'
         WHEN state='HISTORICAL' THEN '2'
 END
|| 
legal_type
||' '
||substr(identifier,3,7)
||upper(legal_name)
from businesses
where legal_type in ('SP','GP');
"""
icbc_df = pd.read_sql(icbc_query, engine)

### Save data to file

In [ ]:
import functools

datestr = datetime.strftime(datetime.now(), '%Y%m%d')
icbc_data_filename = os.path.join(os.getcwd(), 'data', f'icbc_data_{datestr}.txt')
os.makedirs(os.path.dirname(icbc_data_filename), exist_ok=True)

with open(icbc_data_filename, 'w') as f:
    if not icbc_df.empty:
        # Change the string output from right alignment to left alignment.
        formatters = {}
        for li in list(icbc_df.columns):
            maxlen = icbc_df[li].astype(str).str.len().max()
            form = f'{{:<{maxlen}s}}'
            formatters[li] = functools.partial(str.format, form)
        dfAsString = icbc_df.to_string(formatters=formatters, header=False, index=False)
        f.write(dfAsString)